# This case study consists of 1000 marks and the solution file is expected to be submitted in the upcoming session for evaluation.

# Combining Datasets: Merge and Join

One essential feature offered by Pandas is its high-performance, in-memory join and merge operations.
If you have ever worked with databases, you should be familiar with this type of data interaction.
The main interface for this is the ``pd.merge`` function, and we'll see few examples of how this can work in practice.

For convenience, we will start by redefining the ``display()`` functionality from the previous section:

In [1]:
import pandas as pd
import numpy as np


## Specification of the Merge Key

We've already seen the default behavior of ``pd.merge()``: it looks for one or more matching column names between the two inputs, and uses this as the key.
However, often the column names will not match so nicely, and ``pd.merge()`` provides a variety of options for handling this.

## Example: US States Data

Merge and join operations come up most often when combining data from different sources.
Here we will consider an example of some data about US states and their populations.
The data files can be found at http://github.com/jakevdp/data-USstates/:

In [12]:
# Following are shell commands to download the data
! mkdir -p data
!cd data ; curl -O  https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-population.csv ; cd ..
!cd data ; curl -O https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-areas.csv ; cd ..
!cd data ; curl -O https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-abbrevs.csv ; cd ..

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 57935  100 57935    0     0  78182      0 --:--:-- --:--:-- --:--:-- 78184
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   835  100   835    0     0   2553      0 --:--:-- --:--:-- --:--:--  2553
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   872  100   872    0     0   2720      0 --:--:-- --:--:-- --:--:--  2725


In [13]:
!ls

03.06-Concat-And-Append.ipynb  data		      data_exploration.ipynb
03.07-Merge-and-Join.ipynb     Data Cleaning 1.ipynb


Let's take a look at the three datasets, using the Pandas ``read_csv()`` function:

In [6]:
pop = pd.read_csv('https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-population.csv')
areas = pd.read_csv('https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-areas.csv')
abbrevs = pd.read_csv('https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-abbrevs.csv')


Given this information, say we want to compute a relatively straightforward result: rank US states and territories by their 2010 population density.
We clearly have the data here to find this result, but we'll have to combine the datasets to find the result.

We'll start with a many-to-one merge that will give us the full state name within the population ``DataFrame``.
We want to merge based on the ``state/region``  column of ``pop``, and the ``abbreviation`` column of ``abbrevs``.
We'll use ``how='outer'`` to make sure no data is thrown away due to mismatched labels.

In [7]:
pop.head()

,state/region,ages,year,population
0,AL,under18,2012,1117489.0
1,AL,total,2012,4817528.0
2,AL,under18,2010,1130966.0
3,AL,total,2010,4785570.0
4,AL,under18,2011,1125763.0


In [8]:
areas.head()

,state,area (sq. mi)
0,Alabama,52423
1,Alaska,656425
2,Arizona,114006
3,Arkansas,53182
4,California,163707


In [9]:
abbrevs.head()

,state,abbreviation
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA


## Merge the 2 dataframes, pop and abbrevs such that all rows in both tables are included. Drop any duplicate column in the merged dataframe.

Let's double-check whether there were any mismatches here, which we can do by looking for rows with nulls:

In [19]:
merged = pd.merge( pop, abbrevs, left_on = 'state/region',right_on ='abbreviation', how = 'outer')

In [20]:
merged.drop('abbreviation', axis = 1, inplace = True)

In [21]:
merged.head()

,state/region,ages,year,population,state
0,AL,under18,2012,1117489.0,Alabama
1,AL,total,2012,4817528.0,Alabama
2,AL,under18,2010,1130966.0,Alabama
3,AL,total,2010,4785570.0,Alabama
4,AL,under18,2011,1125763.0,Alabama


In [24]:
merged.tail()

,state/region,ages,year,population,state
2539,USA,total,2010,309326295.0,NaN
2540,USA,under18,2011,73902222.0,NaN
2541,USA,total,2011,311582564.0,NaN
2542,USA,under18,2012,73708179.0,NaN
2543,USA,total,2012,313873685.0,NaN


## Look for any rows with null values in the new dataframe

In [23]:
merged.isnull().sum()

state/region     0
ages             0
year             0
population      20
state           96
dtype: int64

## Print out the rows in the merged dataframe that has null values for population.

In [25]:
merged[merged['population'].isnull()]

,state/region,ages,year,population,state
2448,PR,under18,1990,NaN,NaN
2449,PR,total,1990,NaN,NaN
2450,PR,total,1991,NaN,NaN
2451,PR,under18,1991,NaN,NaN
2452,PR,total,1993,NaN,NaN
2453,PR,under18,1993,NaN,NaN
2454,PR,under18,1992,NaN,NaN
2455,PR,total,1992,NaN,NaN
2456,PR,under18,1994,NaN,NaN
2457,PR,total,1994,NaN,NaN


It appears that all the null population values are from Puerto Rico prior to the year 2000; this is likely due to this data not being available from the original source.

More importantly, we see also that some of the new ``state`` entries are also null, which means that there was no corresponding entry in the ``abbrevs`` key!
Let's figure out which regions lack this match:

In [26]:
merged.loc[merged['state'].isnull(), 'state/region'].unique()

array(['PR', 'USA'], dtype=object)

We can quickly infer the issue: our population data includes entries for Puerto Rico (PR) and the United States as a whole (USA), while these entries do not appear in the state abbreviation key.
We can fix these quickly by filling in appropriate entries:

In [27]:
merged.loc[merged['state/region'] == 'PR', 'state'] = 'Puerto Rico'
merged.loc[merged['state/region'] == 'USA', 'state'] = 'United States'
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state           False
dtype: bool

No more nulls in the ``state`` column: we're all set!

Now we can merge the result with the area data using a similar procedure.
Examining our results, we will want to join on the ``state`` column in both:

In [33]:
final = pd.merge(merged, areas, on='state', how='left')
final.head()

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0
2,AL,under18,2010,1130966.0,Alabama,52423.0
3,AL,total,2010,4785570.0,Alabama,52423.0
4,AL,under18,2011,1125763.0,Alabama,52423.0


## Repeat the process of checking for nulls in the area field for each state in the merged dataframe and deal with them appropriately.

In [36]:
final.loc[final['area (sq. mi)'].isnull(), 'state/region'].unique()

array(['USA'], dtype=object)

In [52]:
#final[final['state/region'] == 'USA'].count()
final.loc[final['state/region'] == 'USA', 'area (sq. mi)'] = 3531905

In [53]:
final['area (sq. mi)'].isnull().sum()

0

## Select the portion of the data corresponding with the year 2000, and the total population.

In [71]:
#final.loc[final['year'] == 2000] and final.loc[final['ages'] == 'total']
data2000 =final.query("year == 2000 & ages == 'total' ")
data2000.shape

(53, 7)

## Re-index the data on state and find population density and display by density in ascending order

The result is a ranking of US states plus Washington, DC, and Puerto Rico in order of their 2010 population density, in residents per square mile.
We can see that by far the densest region in this dataset is Washington, DC (i.e., the District of Columbia); among states, the densest is New Jersey.

We can also check the end of the list:

We see that the least dense state, by far, is Alaska, averaging slightly over one resident per square mile.

This type of messy data merging is a common task when trying to answer questions using real-world data sources.
I hope that this example has given you an idea of the ways you can combine tools we've covered in order to gain insight from your data!

In [72]:
data2000.set_index('state', inplace = True)

In [73]:
density = data2000['population'] / data2000['area (sq. mi)']
density

state
Alabama                   84.927856
Alaska                     0.956641
Arizona                   45.265916
Arkansas                  50.366440
California               207.614684
Colorado                  41.565043
Connecticut              615.399892
Delaware                 402.442682
District of Columbia    8412.441176
Florida                  244.038976
Georgia                  138.411248
Hawaii                   111.006129
Idaho                     15.548257
Illinois                 214.685607
Indiana                  167.267051
Iowa                      52.048244
Kansas                    32.737184
Kentucky                 100.196011
Louisiana                 86.258222
Maine                     36.088733
Maryland                 428.067543
Massachusetts            602.662624
Michigan                 102.803946
Minnesota                 56.746282
Mississippi               58.808957
Missouri                  80.438466
Montana                    6.146192
Nebraska              

In [74]:
density.sort_values(ascending = False, inplace = True)

In [75]:
density.head()

state
District of Columbia    8412.441176
Puerto Rico             1084.098151
New Jersey               966.592639
Rhode Island             679.785113
Connecticut              615.399892
dtype: float64

In [76]:
density.tail()

state
South Dakota    9.800755
North Dakota    9.080434
Montana         6.146192
Wyoming         5.053262
Alaska          0.956641
dtype: float64